# Results

## Imports and functions

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import pandas as pd
import os
from enum import Enum

pd.set_option("display.precision", 5)
pd.set_option('display.max_rows', None)

In [2]:
metadata_file = "./results/metadata.pkl"

class dataset_types(Enum):
    train = 1
    development = 2
    test = 3
    
    def title(self):
        shorten_names = ("Train", "Dev", "Test")
        return shorten_names[self.value - 1]

    def __lt__(self, other):
        return self.value < other.value

def save_results(y_pred, index, name, task, language, dataset_type, group=None, description=None, truth=False, filename=None):
    """Auxiliar method to save predictions and extra data through pickle. All extra information will be stored though a metadata file.
    Some atributes will be used to generate the path where the prediction is stored.
    
    The metadata file will store:
    - Path: where the prediction is stored.
    - Name: name of the model.
    - Description: extra information of the prediction. For example, 'the model parameters'.
    - Dataset type: enum value of the types of datasets.
    - Groud Truth: if the path contains the truth and not a prediction.
    - Group: model group (used in path). Example, 'best_traditional'.
    - Task: name of the task (used in path). Example, 'hateval2019/task1'.
    - Language: language of the task (used in path).
    
    - `filename`: name of the file (used in path).
    """
    
    path = f"./results/{task}/{language}/{dataset_type.name}{'/' + group if group is not None else ''}/{name if filename is None else filename}.pkl"
    
    directory = "/".join(path.split("/")[:-1])
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    if os.path.exists(metadata_file):
        metadata = pd.read_pickle(metadata_file)
    else:
        metadata = pd.DataFrame({
            "Path": pd.Series([], dtype=str),
            "Name": pd.Series([], dtype=str),
            "Description": pd.Series([], dtype=str),
            "Dataset type": pd.Categorical([], categories=dataset_types, ordered=False),
            "Groud Truth": pd.Series([], dtype=bool),
            "Group": pd.Series([], dtype=str),
            "Task": pd.Series([], dtype=str),
            "Language": pd.Series([], dtype=str),
        }).set_index("Path")
    
    if path in metadata.index:
        metadata = remove_results(path)

    metadata.loc[path] = {"Name": name, "Description": description, "Dataset type": dataset_type, "Groud Truth": truth, "Group": group, "Task": task, "Language": lenguage}
    results = pd.DataFrame({"id": index, "y_pred": y_pred}).set_index("id") 
    
    results.to_pickle(path)
    metadata.to_pickle(metadata_file)
    
    print("Results saved on: " + path)

def remove_results(path=None):
    """Removes prediction from specified path from the disk and the metadata.
    In case `path is None` removes from the disk all files in './results' that aren't in the metadata.
    """
    # Checks if metadata exists else does nothing
    if os.path.exists(metadata_file):
        metadata = pd.read_pickle(metadata_file)
        # If path removes file from metadata and disk
        if path is not None:
            if os.path.exists(path):
                metadata = metadata.drop(path)
                os.remove(path)
                metadata.to_pickle(metadata_file)
        else:
            # Removes files from disk if they aren't in metadata
            used_files = [os.path.normpath(f) for f in metadata.index]
            all_files = [os.path.normpath(os.path.join(dp, f)) for dp, dn, filenames in os.walk('./results') for f in filenames][1:]
            for f in all_files:
                if f not in set(used_files):
                    print(f)
                    os.remove(f)
        return metadata

    
def load_results():
    """Loades from disk a DataFrame containing all the metadata and the predictions. 
    The predictions are also stored loaded into a DataFrame. 
    In case, no prediction has been stored, returns `None`.    
    """
    if os.path.exists(metadata_file):
        metadata = pd.read_pickle(metadata_file)
    else:
        return None

    # Loads all predictions from disk
    metadata["Prediction"] = [pd.read_pickle(path) for path in metadata.index]
    return metadata

In [3]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

def score(y_true, y_pred, name, f1_average):
    acc = f1 = precision = recall =  None
    if y_true is not None:
        acc, f1 = accuracy_score(y_true, y_pred), f1_score(y_true, y_pred, average=f1_average, zero_division=0)   
        precision, recall = precision_score(y_true, y_pred, zero_division=0), recall_score(y_true, y_pred, zero_division=0)
    
    return {"F1": f1, "Accuracy": acc, "Precision": precision, "Recall": recall}

def score_hateval_task1(y_true, y_pred, name):
    return score(y_true, y_pred, name, 'macro')
    
def score_detoxis_task1(y_true, y_pred, name):
    return score(y_true, y_pred, name, 'binary')

## Loads Results

In [4]:
# Loads dataframe with results
df_results = load_results()

# Finds gold standard of the datasets
mask = df_results["Groud Truth"] == True

# Splits the data into truth and predictions
df_truth = df_results[mask]
df_pred = df_results[~mask]

In [5]:
# List of dictionaries of all predictions
results = []

for path, (name, desc, dataset_type, truth, group, task, language, y_pred) in df_pred.iterrows():
    # Tires to find the gold standard corresping to the current dataset
    y_true = df_truth[(df_truth["Dataset type"] == dataset_type) & (df_truth["Task"] == task) & (df_truth["Language"] == language)]["Prediction"]
    # Checks if truth is empty
    if y_true.empty:
        y_true = [None]
    
    # Checks the task type to find the corresponding scoring method
    if task == "hateval2019/task1":
        result = score_hateval_task1(y_true[0], y_pred, name)
    elif task == "detoxis/task1":
        result = score_detoxis_task1(y_true[0], y_pred, name)
    else:
        raise Exception("Unknown Task")
        
    # Reformats features to better values
    result.update({"Dataset type": dataset_type.title(),
                   "Task": ' '.join(task.split('/')).title(),
                   "Name": name,
                   "Group": ' '.join(group.split('_')).title(),
                   "Language": language.title(),
                   "Description": desc})
    
    results.append(result)

# Generates base DataFrame model
df_results = pd.DataFrame(results).set_index(["Task", "Language", "Dataset type", "Group", "Name", "Description"]).sort_index(level=[0, 1, 2, 3, 4])

## Option 1

Scores group by task, language and dataset type. Also are sorted by F1. Seems the least useful.

In [6]:
score_names = df_results.columns.to_list()
df_results_1 = df_results.sort_values(by=["Task", "Language", "Dataset type"] + score_names, ascending=3*[True] + len(score_names) * [False]).droplevel("Group")
df_results_1.droplevel("Description")

F1  \
Task              Language Dataset type Name                                                 
Detoxis Task1     Spanish  Dev          Bert base (2 epochs)                       0.66368   
                                        RidgeClassifier SMOTE_IPF                  0.66160   
                                        RidgeClassifier SMOTE_TomekLinks           0.65909   
                                        RidgeClassifier G_SMOTE                    0.65774   
                                        RidgeClassifier SMOBD                      0.65774   
                                        RidgeClassifier Assembled_SMOTE            0.65655   
                                        Bert base (3 epochs)                       0.65471   
                                        RidgeClassifier polynom_fit_SMOTE          0.64916   
                                        Bert Avarage                               0.64819   
                                        Bert Avarage (2 epochs| differnt lr)       0.64775   
                                        MLPClassifier G_SMOTE                      0.64552   
                                        MLPClassifier SMOTE_TomekLinks             0.64167   
                                        Bert base (4 epochs)                       0.64069   
                                        MLPClassifier SMOBD                        0.63813   
                                        RidgeClassifier ProWSyn                    0.63359   
                                        MLPClassifier Assembled_SMOTE              0.62992   
                                        Bert Avarage (3 epochs)                    0.62882   
                                        MLPClassifier ProWSyn                      0.62813   
                                        SVC Assembled_SMOTE                        0.62802   
                                        SVC SMOTE_TomekLinks                       0.62350   
                                        SVC SMOBD                                  0.62019   
                                        Multi-layer Perceptron classifier          0.61860   
                                        SVC G_SMOTE                                0.61765   
                                        SVC SMOTE_IPF                              0.61538   
                                        MLPClassifier SMOTE_IPF                    0.61242   
                                        Multi-layer Perceptron classifier (best)   0.60849   
                                        Support Vector Classification (best)       0.60325   
                                        MLPClassifier LVQ_SMOTE                    0.60251   
                                        MLPClassifier polynom_fit_SMOTE            0.59780   
                                        RidgeClassifier LVQ_SMOTE                  0.59556   
                                        SVC ProWSyn                                0.58853   
                                        Ridge Classifier                           0.58621   
                                        Ridge Classifier (best)                    0.57757   
                                        SVC LVQ_SMOTE                              0.56919   
                                        Support Vector Classification              0.56533   
                                        AdaBoost classifier (best)                 0.55530   
                                        SVC polynom_fit_SMOTE                      0.55467   
                                        Dummy Classifier (best)                    0.52396   
                                        AdaBoost classifier (best)                 0.52081   
                                        AdaBoost classifier                        0.50115   
                                        Random Forest classifier (best)            0.41604   
                                        RidgeC

## Option 2

Scores group by task, language and group and sorted by F1 test (if exists) or F1 Dev. Allows to compare model in the same group.

In [7]:
df_results_2 = df_results.groupby(["Task", "Language",'Group','Name'], as_index=False).aggregate([(lambda t: lambda x: x[:, :, t] if x.index.isin([t], level=2).any() else None)(t.title()) for t in dataset_types])
df_results_2.columns = pd.MultiIndex.from_product([score_names, [t.title() for t in dataset_types]])
df_results_2 = df_results_2.sort_values(by=['Task', 'Language', 'Group', ("F1", dataset_types.test.title()), ("F1", dataset_types.development.title())], ascending=3*[True] + 2*[False])

df_results_2

F1  \
                                                                                           Train   
Task              Language Group              Name                                                 
Detoxis Task1     Spanish  Deep Learning      Bert base (2 epochs)                       0.83636   
                                              Bert base (3 epochs)                       0.85899   
                                              Bert Avarage                               0.98041   
                                              Bert base (4 epochs)                       0.95833   
                                              Bert Avarage (2 epochs| differnt lr)       0.86458   
                                              Bert Avarage (3 epochs)                    0.94378   
                                              Atalaya                                    0.23953   
                           Sbert              Multi-layer Perceptron classifier          0.66381   
                                              Ridge Classifier                           0.62394   
                                              Support Vector Classification              0.82534   
                                              AdaBoost classifier                        0.62401   
                                              Random Forest classifier                   0.99944   
                           Sbert Best         Multi-layer Perceptron classifier (best)   0.61733   
                                              Ridge Classifier (best)                    0.64695   
                                              Support Vector Classification (best)       0.99833   
                                              AdaBoost classifier (best)                 0.90189   
                                              Random Forest classifier (best)            0.85611   
                           Sbert Oversampling MLPClassifier G_SMOTE                      0.73046   
                                              MLPClassifier SMOTE_TomekLinks             0.93829   
                                              RidgeClassifier SMOTE_TomekLinks           0.69412   
                                              RidgeClassifier polynom_fit_SMOTE          0.69386   
                                              RidgeClassifier SMOTE_IPF                  0.69382   
                                              RidgeClassifier G_SMOTE                    0.69460   
                                              MLPClassifier SMOBD                        0.68076   
                                              MLPClassifier Assembled_SMOTE              0.73429   
                                              RidgeClassifier SMOBD                      0.68461   
                                              RidgeClassifier ProWSyn                    0.69046   
                                              MLPClassifier LVQ_SMOTE                    0.63343   
                                              MLPClassifier SMOTE_IPF                    0.95996   
                                              MLPClassifier ProWSyn                      0.76596   
                                              RidgeClassifier Assembled_SMOTE            0.69200   
                                              RidgeClassifier LVQ_SMOTE                  0.63524   
                                              MLPClassifier polynom_fit_SMOTE            0.83764   
                                              SVC G_SMOTE                                0.91151   
                                              SVC SMOBD                                  0.90656   
                                              SVC SMOTE_TomekLinks                       0.90951   
                                              SVC SMOTE_IPF                              0.91442   
                                              SVC ProWSyn                                0.907

## Option 3
Scores group by task, language and are sorted by F1 test (if exists) or F1 Dev. Allows to find best models.

In [8]:
df_results_3 = df_results_2.sort_values(by=["Task", "Language", ("F1", dataset_types.test.title()), ("F1", dataset_types.development.title())], ascending=2*[True] + 2*[False])
df_results_3

F1  \
                                                                                           Train   
Task              Language Group              Name                                                 
Detoxis Task1     Spanish  Deep Learning      Bert base (2 epochs)                       0.83636   
                                              Bert base (3 epochs)                       0.85899   
                                              Bert Avarage                               0.98041   
                           Sbert Oversampling MLPClassifier G_SMOTE                      0.73046   
                                              MLPClassifier SMOTE_TomekLinks             0.93829   
                                              RidgeClassifier SMOTE_TomekLinks           0.69412   
                                              RidgeClassifier polynom_fit_SMOTE          0.69386   
                                              RidgeClassifier SMOTE_IPF                  0.69382   
                                              RidgeClassifier G_SMOTE                    0.69460   
                                              MLPClassifier SMOBD                        0.68076   
                                              MLPClassifier Assembled_SMOTE              0.73429   
                                              RidgeClassifier SMOBD                      0.68461   
                                              RidgeClassifier ProWSyn                    0.69046   
                                              MLPClassifier LVQ_SMOTE                    0.63343   
                           Deep Learning      Bert base (4 epochs)                       0.95833   
                           Sbert Oversampling MLPClassifier SMOTE_IPF                    0.95996   
                           Deep Learning      Bert Avarage (2 epochs| differnt lr)       0.86458   
                           Sbert Oversampling MLPClassifier ProWSyn                      0.76596   
                                              RidgeClassifier Assembled_SMOTE            0.69200   
                           Deep Learning      Bert Avarage (3 epochs)                    0.94378   
                           Sbert Best         Multi-layer Perceptron classifier (best)   0.61733   
                           Sbert Oversampling RidgeClassifier LVQ_SMOTE                  0.63524   
                                              MLPClassifier polynom_fit_SMOTE            0.83764   
                           Sbert              Multi-layer Perceptron classifier          0.66381   
                           Sbert Oversampling SVC G_SMOTE                                0.91151   
                                              SVC SMOBD                                  0.90656   
                                              SVC SMOTE_TomekLinks                       0.90951   
                                              SVC SMOTE_IPF                              0.91442   
                                              SVC ProWSyn                                0.90792   
                           Sbert              Ridge Classifier                           0.62394   
                           Sbert Best         Ridge Classifier (best)                    0.64695   
                           Sbert Oversampling SVC LVQ_SMOTE                              0.85162   
                           Sbert Best         Support Vector Classification (best)       0.99833   
                           Sbert Oversampling SVC Assembled_SMOTE                        0.90487   
                           Traditional Best   AdaBoost classifier (best)                 0.49262   
                                              Dummy Classifier (best)                    0.49087   
                           Sbert              Support Vector Classification              0.82534   
                           Sbert Oversampling SVC polynom_fit_SMOTE                      0.866

## Option 4
Same as option 2, but with reduced number of models

In [9]:
df_results_4 = df_results_2.sort_values(by=["Task", "Language", "Group", ("F1", dataset_types.test.title()), ("F1", dataset_types.development.title())], ascending=3*[True] + 2*[False]).groupby(by=["Task", "Language","Group"]).head(3)
df_results_4

F1  \
                                                                                           Train   
Task              Language Group              Name                                                 
Detoxis Task1     Spanish  Deep Learning      Bert base (2 epochs)                       0.83636   
                                              Bert base (3 epochs)                       0.85899   
                                              Bert Avarage                               0.98041   
                           Sbert              Multi-layer Perceptron classifier          0.66381   
                                              Ridge Classifier                           0.62394   
                                              Support Vector Classification              0.82534   
                           Sbert Best         Multi-layer Perceptron classifier (best)   0.61733   
                                              Ridge Classifier (best)                    0.64695   
                                              Support Vector Classification (best)       0.99833   
                           Sbert Oversampling MLPClassifier G_SMOTE                      0.73046   
                                              MLPClassifier SMOTE_TomekLinks             0.93829   
                                              RidgeClassifier SMOTE_TomekLinks           0.69412   
                           Traditional        Ridge Classifier                           0.82495   
                                              Bernoulli Naive Bayes classifier           0.66345   
                                              Random Forest classifier                   0.97798   
                           Traditional Best   AdaBoost classifier (best)                 0.49262   
                                              Dummy Classifier (best)                    0.49087   
                                              Random Forest classifier (best)            0.72641   
Hateval2019 Task1 English  Bert Avarage       CLS layers                                 0.93160   
                                              Entire layer                               0.94064   
                                              Tokens                                     0.93611   
                           Deep Learning      Bert SMOTE_TomekLinks                      0.93527   
                                              Bert base                                  0.92999   
                                              Bert Avarage                               0.93938   
                           Sbert              Random Forest classifier                   0.99897   
                                              Support Vector Classification              0.89996   
                                              AdaBoost classifier                        0.75236   
                           Sbert Best         Random Forest classifier (best)            0.99897   
                                              Support Vector Classification (best)       0.99897   
                                              AdaBoost classifier (best)                 0.80969   
                           Sbert Oversampling SVC polynom_fit_SMOTE                      0.91803   
                                              SVC CCR                                    0.89754   
                                              SVC LVQ_SMOTE                              0.90041   
                           Traditional        Multi-layer Perceptron classifier          0.78679   
                                              Ridge Classifier                           0.87636   
                                              Multinomial Naive Bayes classifier         0.81284   
                           Traditional Best   Multi-layer Perceptron classifier (best)   0.81200   
                                              Multinomial Naive Bayes classifier (best)  0.816

## Option 5
Same as option 4, but without restriting groups

In [10]:
df_results_5 = df_results_2.sort_values(by=["Task", "Language", ("F1", dataset_types.test.title()), ("F1", dataset_types.development.title())], ascending=2*[True] + 2*[False]).groupby(by=["Task", "Language","Group"]).head(3)
df_results_5

F1  \
                                                                                           Train   
Task              Language Group              Name                                                 
Detoxis Task1     Spanish  Deep Learning      Bert base (2 epochs)                       0.83636   
                           Sbert Oversampling RidgeClassifier SMOTE_IPF                  0.69382   
                                              RidgeClassifier SMOTE_TomekLinks           0.69412   
                                              RidgeClassifier G_SMOTE                    0.69460   
                           Deep Learning      Bert base (3 epochs)                       0.85899   
                                              Bert Avarage                               0.98041   
                           Sbert              Multi-layer Perceptron classifier          0.66381   
                           Sbert Best         Multi-layer Perceptron classifier (best)   0.61733   
                                              Support Vector Classification (best)       0.99833   
                           Sbert              Ridge Classifier                           0.62394   
                           Sbert Best         Ridge Classifier (best)                    0.64695   
                           Sbert              Support Vector Classification              0.82534   
                           Traditional Best   Dummy Classifier (best)                    0.49087   
                                              AdaBoost classifier (best)                 0.49262   
                                              Random Forest classifier (best)            0.72641   
                           Traditional        Ridge Classifier                           0.82495   
                                              Bernoulli Naive Bayes classifier           0.66345   
                                              AdaBoost classifier                        0.30435   
Hateval2019 Task1 English  Bert Avarage       Entire layer                               0.94064   
                           Deep Learning      Bert Avarage                               0.93938   
                           Bert Avarage       CLS layers                                 0.93160   
                           Deep Learning      GPT2 base                                  0.83095   
                           Bert Avarage       Tokens                                     0.93611   
                           Deep Learning      Bert SMOTE_TomekLinks                      0.93527   
                           Traditional        Random Forest classifier                   0.99897   
                           Sbert Oversampling SVC ProWSyn                                0.90737   
                           Traditional Best   Random Forest classifier (best)            0.99772   
                           Sbert Oversampling SVC SMOBD                                  0.90641   
                           Traditional Best   Bernoulli Naive Bayes classifier (best)    0.81365   
                           Sbert Oversampling MLPClassifier SMOBD                        0.88647   
                           Traditional        Bernoulli Naive Bayes classifier           0.81929   
                                              Multinomial Naive Bayes classifier         0.81284   
                           Traditional Best   Multinomial Naive Bayes classifier (best)  0.81622   
                           Sbert              Support Vector Classification              0.89996   
                                              Multi-layer Perceptron classifier          0.87029   
                                              Ridge Classifier                           0.80368   
                           Sbert Best         Ridge Classifier (best)                    0.80402   
                                              Multi-layer Perceptron classifier (best)   0.783